```@meta
CurrentModule = AlgorithmicRecourse 
```

# From cat to dog

In [161]:
using Plots, AlgorithmicRecourse, Random
using Logging
disable_logging(Logging.Info)

LogLevel(1)

In [186]:
retrain = false
include("CIFAR10.jl")
println("Training accuracy: " * string(accuracy(train_x, train_y)))
println("Validation accuracy: " * string(accuracy(val_x, val_y)))

Training accuracy: 0.8423333333333334


Validation accuracy: 0.736


In [187]:
using AlgorithmicRecourse, AlgorithmicRecourse.Models
import AlgorithmicRecourse.Models: logits, probs # import functions in order to extend

# Step 1)
struct NeuralNetwork <: Models.FittedModel
    nn::Any
end

# Step 2)
logits(𝑴::NeuralNetwork, X::AbstractArray) = 𝑴.nn(X)
probs(𝑴::NeuralNetwork, X::AbstractArray)= σ.(logits(𝑴, X))
𝑴 = NeuralNetwork(nn);

In [194]:
ŷ = vec(round.(probs(𝑴, train_x))) # predicted class
# ŷ = CIFAR10.trainlabels()[1:1000]
cats = findall(ŷ .== 0)
dogs = findall(ŷ .== 1)
# Plot random samples:
using Plots
Random.seed!(123)
n_samples = 10
using StatsBase
random_cats = sample(cats, n_samples, replace=false)
random_dogs = sample(dogs, n_samples, replace=false)
p_list = [plot(conv2image(train_x[:,:,:,i]), size=(100,100), axis=nothing, title=ifelse(train_y[i]==ŷ[i],"Correct","Wrong")) for i ∈ vcat(random_cats, random_dogs)];

In [196]:
plt = plot(p_list...,layout=(2,n_samples), size=(1000,200))
savefig(plt, "www/CIFAR10_predictions.png")

![](www/CIFAR10_predictions.png)